# Image Segmentation
It is the partitioning of images into different regions 


# Contours
They are continuous lines or curves that bound or cover the full boundary of an object in an image.


## Useful in :
1. Object Detection
2. Shape Analysis


In [2]:
import cv2
import numpy as np

image = cv2.imread('./images/input.jpg')
cv2.imshow('Input Image', image)
cv2.waitKey(0)
##Convert to gray scale. Colored images won't work  with openCV
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

### Find canny edges, it is not necessary, but good to reduce the number of unnecessary contours when doing findcontours
edged = cv2.Canny(gray, 30, 200)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

##Finding Contours
## Use a copy of your image e.g. edged.copy(), since findCountours alters the image
_, contours, hierarchy = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)  ## Change cv2.RETR_EXTERNAL to cv2.RETR_LIST
cv2.imshow('Canny Edges after Contouring', edged)
cv2.waitKey(0)

print('Number of Contours found = ' + str(len(contours)))

# Draw all Contours
## use '-1' as the 3rd parameter to draw all
cv2.drawContours(image, contours, -1, (0,255,0), 3)

cv2.imshow('Contours', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



Number of Contours found = 63


### Retrieval Modes:
###CV_RETR_EXTERNAL retrieves only the extreme outer contours. It sets hierarchy[i][2]=hierarchy[i][3]=-1 for all the contours.
###CV_RETR_LIST retrieves all of the contours without establishing any hierarchical relationships.
###CV_RETR_CCOMP retrieves all of the contours and organizes them into a two-level hierarchy. At the top level, there are external boundaries of the components. At the second level, there are boundaries of the holes. If there is another contour inside a hole of a connected component, it is still put at the top level.
###CV_RETR_TREE retrieves all of the contours and reconstructs a full hierarchy of nested contours. This full hierarchy is built and shown in the OpenCV contours.c demo.
######method –
###Contour approximation method (if you use Python see also a note below).
### Approximation modes
###CV_CHAIN_APPROX_NONE stores absolutely all the contour points. That is, any 2 subsequent points (x1,y1) and (x2,y2) of the contour will be either horizontal, vertical or diagonal neighbors, that is, max(abs(x1-x2),abs(y2-y1))==1.
###CV_CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments and leaves only their end points. For example, an up-right rectangular contour is encoded with 4 points.
###CV_CHAIN_APPROX_TC89_L1,CV_CHAIN_APPROX_TC89_KCOS applies one of the flavors of the Teh-Chin chain approximation algorithm. See [TehChin89] for details.
###offset – Optional offset by which every contour point is shifted. This is useful if the contours are extracted from the image ROI and then they should be analyzed in the whole image context.

# Sorting Contours
It is quite useful when doing image processing.
Sorting by area can assist in object recognition (using contour area)
1. Eliminate small contours that nay be noise
    OR
2. Extract the largest contour

# Sorting by Spatial Position (using the contour centroid)
1. Sort characters left to right
    OR
2. Process images in specific order



In [3]:
import cv2
import numpy as np
# Load image
image=cv2.imread('./images/shapes.jpg')
cv2.imshow("0-Original Image",image)
cv2.waitKey(0)

#create a black image with same dimensions as our loaded image

black_image=np.zeros((image.shape[0],image.shape[1],3))
#print(black_image)
#create a copy of our original image

original_image=image

#Grayscale our image
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

#Find Canny edges

canny=cv2.Canny(gray,50,200)
cv2.imshow("1 - Canny Edges",canny)
cv2.waitKey(0)

#Find Contours and print how many were found

_,contours,hierarchy=cv2.findContours(canny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
print("Number of contours found = ",len(contours))

#Draw all contours over black_image

cv2.drawContours(black_image,contours,-1,(0,255,0),3)
cv2.imshow('2 - All Contours over black-image',black_image)
cv2.waitKey(0)

#Draw all contours over image

cv2.drawContours(image,contours,-1,(0,255,0),3)
cv2.imshow("3 - All Contours",image)
cv2.waitKey(0)
cv2.destroyAllWindows()



Number of contours found =  3


# Lets now sort by area


In [5]:
import cv2
import numpy as np

def get_contour_areas(contours):
    #returns the areas of all contours as a list
    all_areas=[]
    for cnt in contours:
        area=cv2.contourArea(cnt)
        all_areas.append(area)
        return all_areas
    
# Load image
image=cv2.imread('./images/shapes.jpg')
original_image=image

#Lets print the areas of the contours before sorting
print("Contour areas before sorting")
print (get_contour_areas(contours))

#Sort contours large to small

sorted_contours= sorted(contours,key=cv2.contourArea,reverse=True)

print("Contour areas after sorting")
print (get_contour_areas(contours))

#Iterate over our contour's and draw one at a time
for c in sorted_contours:
    cv2.drawContours(original_image,[c],-1,(255,0,0),3)
    cv2.waitKey(0)
    
cv2.imshow("Contours by area",original_image)
cv2.waitKey(0)

cv2.destroyAllWindows()



Contour areas before sorting
[10751.5]
Contour areas after sorting
[10751.5]


# Approximating Contours and finding their convex hull.

cv2.approxPolyDP(contour,Approximation accuracy, closed)
1. Contour - is the individual contour we wish to approximate
2. Approximation Accuracy - Important Parameter is determining the accuracy of the approximation. Small values give precise- approximations, large values give more generic approximation. A good rule of thumb is less than 5% of the contour perimeter.

3. Closed - a boolean value that states whether the approximate contour should be opened or closed.   

In [ ]:

import numpy as np
import cv2

# Load image and keep a copy

image=cv2.imread('./images/house.jpg')
orig_image= image.copy()
cv2.imshow("Original image",orig_image)
cv2.waitKey(0)

#Grayscale and binarize

gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127,255,cv2.THRESH_BINARY_INV)

#Find Contours

contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

#Iterate through each contour and compute the bounding rectangle 

for c in counters:
    x,y,w,h=cv2.boundingRect(c)
    cv2.rectangle(orig_image,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.imshow("Bounding Rectangle", orig_image)
    cv2.waitKey(0)
    
#Iterate through each contour and compute the bounding rectangle 

for c in contours:
    accuracy=0.03*cv2.arcLength(c,True)
    approx=cv2.approxPolyDP(c,accuracy,true)
    cv2.drawContours(image,[approx],0,(0,255,0),2)
    cv2.imshow("Approx Poly DP",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Convex Hull



In [1]:
import numpy as np
import cv2

# Load image and keep a copy

image=cv2.imread('./images/house.jpg')
orig_image= image.copy()
gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow('Original Image',orig_image)
cv2.waitKey(0)

#Threshold the image
ret, thresh = cv2.threshold(gray, 127,255,cv2.THRESH_BINARY_INV)

#Find Contours

contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

#Sort the contours by area and then remove the largest frame contour

n= len (contours) -1
contours=sorted(contours,key=cv2.contourArea,reverse=False)[:n]

#Iterate through contours and draw the convex hull

for c in contours:
    
    hull=cv2.convexHull(c)
    cv2.drawContours(image,[hull],0,(0,255,0),2)
    cv2.imshow('Convex Hull', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    





ValueError: too many values to unpack (expected 2)